In [7]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import os

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

from metal.end_model import EndModel
from metal.mmtl.dataset import QNLIDataset
from metal.mmtl.modules import BertEncoder
from metal.mmtl.metal_model import MetalModel
from metal.mmtl.scorer import Scorer
from metal.mmtl.task import Task
from metal.mmtl.trainer import MultitaskTrainer

### Config

In [9]:
bert_model = 'bert-base-uncased'
bert_model_output_shape = 768
max_len = 512
batch_size = 32
split_prop = 0.8
trainer_config = {
    "verbose": True,
    "device": "cuda",
    "loss_fn_reduction": "mean",
    "progress_bar": True,
    #"data_loader_config": {"batch_size": 32, "num_workers": 1, "shuffle": True}, ## TODO? 
    "n_epochs": 1,
    # 'grad_clip': 1.0,  ## TODO? 
    "l2": 0.01,
    "validation_metric": "accuracy",
    "validation_freq": 1,
    "validation_scoring_kwargs": {},
    "optimizer_config": {
        "optimizer": "adam",
        "optimizer_common": {"lr": 1e-5},
        "adam_config": {"betas": (0.9, 0.999)},
    },
    "lr_scheduler": "exponential", # reduce_on_plateau  ## TODO? Warmup
    "lr_scheduler_config": {
        "lr_freeze": 0,
        # Scheduler - exponential
        "exponential_config": {"gamma": 0.9},  # decay rate
        # Scheduler - reduce_on_plateau
        "plateau_config": {
            "factor": 0.5,
            "patience": 10,
            "threshold": 0.0001,
            "min_lr": 1e-4,
        },
    },
    # Logger (see metal/logging/logger.py for descriptions)
    "logger": True,
    "logger_config": {
        "log_unit": "epochs",  # ['seconds', 'examples', 'batches', 'epochs']
        "log_train_every": 1,  # How often train metrics are calculated (optionally logged to TB)
        "log_train_metrics": [
            "loss",
        ],  # Metrics to calculate and report every `log_train_every` units. This can include built-in and user-defined metrics.
        "log_train_metrics_func": None,  # A function or list of functions that map a model + train_loader to a dictionary of custom metrics
        "log_valid_every": 1,  # How frequently to evaluate on valid set (must be multiple of log_freq)
        "log_valid_metrics": [
            "accuracy"
        ],  # Metrics to calculate and report every `log_valid_every` units; this can include built-in and user-defined metrics
        "log_valid_metrics_func": None,  # A function or list of functions that maps a model + valid_loader to a dictionary of custom metrics
    },
    # Checkpointer (see metal/logging/checkpointer.py for descriptions)
    "checkpoint": False,  # If True, checkpoint models when certain conditions are met
    "checkpoint_config": {
        "checkpoint_best": True,
        "checkpoint_every": None,  # uses log_valid_unit for units; if not None, checkpoint this often regardless of performance
        "checkpoint_metric": "accuracy",  # Must be in metrics dict; assumes valid split unless appended with "train/"
        "checkpoint_metric_mode": "max",  # ['max', 'min']
        "checkpoint_dir": "checkpoints/qnli_single",
        "checkpoint_runway": 0,
    },
}

In [ ]:
dataloaders = {}
train_ds = QNLIDataset(split='dev', bert_model=bert_model, max_len=max_len)
train_dl, dev_dl = train_ds.get_dataloader(split_prop=split_prop, batch_size=32)
test_ds = QNLIDataset(split='dev', bert_model='bert-base-uncased', max_len=512)
test_dl = test_ds.get_dataloader(batch_size=32)
dataloaders = {
    'train': train_dl,
    'dev': dev_dl,
    'test': test_dl
}

In [5]:
bert_encoder = BertEncoder(bert_model)

In [14]:
from functools import partial
from typing import Callable, List
import torch.nn.functional as F

ranking_head = nn.Linear(in_features=bert_model_output_shape, out_features=2, bias=False)
ranking_task = Task(
    name="ranking",
    data_loaders=[dataloaders['dev'], dataloaders['dev'], dataloaders['test']], # TODO: add dict
    input_module=bert_encoder,
    head_module=ranking_head,
    scorer=Scorer(standard_metrics=["accuracy"]),
    loss_hat_func= lambda X, Y: F.cross_entropy(X, Y - 1),
    output_hat_func=partial(F.softmax, dim=1)
)

In [6]:
tasks = [ranking_task]
model = MetalModel(tasks, verbose=False)
trainer = MultitaskTrainer()
trainer.train_model(
    model,
    tasks,
    **trainer_config)

cpu
cuda
Using GPU...
cuda
cuda


HBox(children=(IntProgress(value=0, max=35), HTML(value='')))

cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda
cuda



/afs/cs.stanford.edu/u/chami/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:728: UserWarning: Attempting to work in a virtualenv. If you encounter problems, please install IPython inside the virtualenv.
  warn("Attempting to work in a virtualenv. If you encounter problems, please "


Python 3.6.7 | packaged by conda-forge | (default, Nov 21 2018, 03:09:43) 
Type 'copyright', 'credits' or 'license' for more information
IPython 6.2.1 -- An enhanced Interactive Python. Type '?' for help.

In [1]: self.config['device']
Out[1]: 'cuda'

In [2]: model.is_cida
---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
/dfs/scratch0/chami/metal/metal/mmtl/trainer.py in <module>()
----> 1 model.is_cida

/dfs/scratch0/chami/miniconda3/envs/metal/lib/python3.6/site-packages/torch/nn/modules/module.py in __getattr__(self, name)
    533                 return modules[name]
    534         raise AttributeError("'{}' object has no attribute '{}'".format(
--> 535             type(self).__name__, name))
    536 
    537     def __setattr__(self, name, value):

AttributeError: 'MetalModel' object has no attribute 'is_cida'

In [3]: model.is_cuda
-----------------------------------------------

In [5]: model.config
Out[5]: {'device': 'cpu', 'seed': None, 'verbose': False}

In [6]: self.config
Out[6]: 
{'checkpoint': False,
 'checkpoint_config': {'checkpoint_best': True,
  'checkpoint_dir': 'checkpoints/qnli_single',
  'checkpoint_every': None,
  'checkpoint_metric': 'accuracy',
  'checkpoint_metric_mode': 'max',
  'checkpoint_runway': 0},
 'data_loader_config': {'batch_size': 32, 'num_workers': 1, 'shuffle': True},
 'device': 'cuda',
 'l2': 0.01,
 'logger': True,
 'logger_config': {'log_train_every': 1,
  'log_train_metrics': ['loss'],
  'log_train_metrics_func': None,
  'log_unit': 'epochs',
  'log_valid_every': 1,
  'log_valid_metrics': ['accuracy'],
  'log_valid_metrics_func': None},
 'loss_fn_reduction': 'mean',
 'loss_weights': None,
 'lr_scheduler': 'exponential',
 'lr_scheduler_config': {'exponential_config': {'gamma': 0.9},
  'lr_freeze': 0,
  'plateau_config': {'factor': 0.5,
   'min_lr': 0.0001,
   'patience': 10,
   'threshold': 0.0001}},
 'n_epochs': 1,
 'optimize

KeyboardInterrupt: 

In [ ]:
for (X, Y) in dataloaders['dev']:
    print(model(X, ['ranking']))
    print(model.calculate_loss(X, Y, ['ranking']))    
    print(model.calculate_output(X, ['ranking']))    
    break